In [1]:
import pandas as pd
import iotbx.pdb
import sys
import numpy as np
import pickle
import math
import matplotlib.pylab as plt
import time
import scipy as sp
import cctbx
import seaborn as sns
from scipy.optimize import fmin, fmin_cg, leastsq, fmin_l_bfgs_b
from scipy.interpolate import interp1d
import scipy.stats 
sns.set_palette("colorblind")
import requests
import re

In [2]:
df = pd.DataFrame()
def read_data(struct):
    rows=[]
    st = iotbx.pdb.input(file_name=struct)
    for atm in st.atoms_with_labels():
        if not atm.element==" H":
            rows.append({ 'b':atm.b,
                          'x': atm.xyz[0],
                          'y': atm.xyz[1],
                          'z': atm.xyz[2],
                          'chain': atm.chain_id,
                          'resname': atm.resname,
                          'Residue number': int(atm.resseq),
                          'combo': atm.icode+atm.resseq+atm.altloc,
                          'altloc': atm.altloc,
                          'Residue_label': atm.icode+atm.resseq+atm.altloc+' '+atm.resname,
                          'Residue_label2': atm.icode+atm.resseq+' '+atm.resname,
                          'atmname': atm.name })
    return pd.DataFrame(rows)

In [5]:


def get_uniprot_entries(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_data = response.text
    
    uniprot_entries = []
    
    for line in pdb_data.split('\n'):
        if line.startswith('DBREF') and 'UNP' in line:
            match = re.search(r'UNP\s+(\w+)', line)
            if match:
                uniprot_entry = match.group(1)
                chain_id = line[12]
                uniprot_entries.append((chain_id, uniprot_entry))
    
    return uniprot_entries

# Example usage
pdb_id = '6QHD'  # Replace with your desired PDB ID
pdb_id_sc = '6qhd'
uniprot_entries = get_uniprot_entries(pdb_id)
chdict={}
entries=[]
# Print the retrieved chains and UniProt entries
for chain_id, uniprot_entry in uniprot_entries:
    print(f"Chain {chain_id}: UniProt Entry {uniprot_entry}")
    chdict[chain_id]=uniprot_entry
    entries.append(uniprot_entry)
entries= list(set(entries))
entries


Chain A: UniProt Entry P40763
Chain B: UniProt Entry P40763


['P40763']

In [6]:
df=read_data(pdb_id_sc+".pdb")
#df=read_data("model_20.pdb")

In [7]:
for key in chdict:
    df.loc[df.chain==key,'Entry']=chdict[key]



In [8]:
df

,b,x,y,z,chain,resname,Residue number,combo,altloc,Residue_label,Residue_label2,atmname,Entry
0,145.04,1.089,-54.170,1.288,A,VAL,136,136,,136 VAL,136 VAL,N,P40763
1,144.24,0.987,-52.668,1.359,A,VAL,136,136,,136 VAL,136 VAL,CA,P40763
2,140.47,1.721,-52.129,2.608,A,VAL,136,136,,136 VAL,136 VAL,C,P40763
3,137.02,1.351,-51.096,3.160,A,VAL,136,136,,136 VAL,136 VAL,O,P40763
4,143.70,-0.499,-52.150,1.275,A,VAL,136,136,,136 VAL,136 VAL,CB,P40763
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9449,105.65,-36.227,-61.246,4.229,D,HOH,1101,1101,,1101 HOH,1101 HOH,O,NaN
9450,92.65,-43.666,-57.997,-1.639,D,HOH,1102,1102,,1102 HOH,1102 HOH,O,NaN
9451,73.96,-43.302,-47.905,-3.910,D,HOH,1103,1103,,1103 HOH,1103 HOH,O,NaN
9452,100.47,-42.611,-39.635,7.459,D,HOH,1104,1104,,1104 HOH,1104 HOH,O,NaN


In [7]:
# Do not use this!
rows=[]
for entry in entries:
    posres=np.load(entry+'.npz')
    posbind=posres['arr_0']
    for x in posbind.T:
            rows.append({ 'Entry': entry,
                          'Residue number':x[0],
                          'robbind': x[1]
                        })
tf=pd.DataFrame(rows)
tf

,Entry,Residue number,robbind
0,P62799,1.0,0.40
1,P62799,2.0,0.60
2,P62799,3.0,0.45
3,P62799,4.0,0.10
4,P62799,5.0,0.45
...,...,...,...
8125,Q6PI79,131.0,0.30
8126,Q6PI79,132.0,0.25
8127,Q6PI79,133.0,0.30
8128,Q6PI79,134.0,0.10


In [9]:
# Use this!
tfs=[]
for entry in entries:
    _tf=pd.read_excel(entry+'_robbind.xlsx')
    tfs.append(_tf)
tf=pd.concat(tfs)
tf['Residue number']=tf['Position_fasta']
tf

,Unnamed: 0,Position_fasta,Entry,robbind,resname_fasta,Residue number
0,0,1,P40763,0.50,M,1
1,1,2,P40763,0.45,A,2
2,2,3,P40763,0.40,Q,3
3,3,4,P40763,0.10,W,4
4,4,5,P40763,0.30,N,5
...,...,...,...,...,...,...
765,765,766,P40763,0.25,A,766
766,766,767,P40763,0.25,T,767
767,767,768,P40763,0.25,S,768
768,768,769,P40763,0.25,P,769


In [10]:
tdf=df[['Entry','chain','Residue number','resname']].drop_duplicates()
cdf=pd.merge(tdf, tf, on=['Entry', 'Residue number'])
cdf=cdf.sort_values(['chain','Residue number'])
cdf.to_excel('cdf_to_check.xlsx')

In [11]:
cdf=pd.read_excel('cdf_checked.xlsx')
#mf = pd.merge(df, tf, on=['Entry', 'Residue number'])
mf = pd.merge(df, cdf, on=['Entry', 'Residue number'])
mf['chain']=mf['chain_x']
mf.to_excel('mf_to_check.xlsx')

In [10]:
mf

,b,x,y,z,chain_x,resname_x,Residue number,combo,altloc,Residue_label,...,atmname,Entry,Unnamed: 0,chain_y,resname_y,Unnamed: 0.1,Position_fasta,robbind,resname_fasta,chain
0,141.98,2.507,12.136,56.214,B,VAL,136,136,,136 VAL,...,N,P40763,1,A,VAL,135,136,0.000000,V,B
1,141.98,2.507,12.136,56.214,B,VAL,136,136,,136 VAL,...,N,P40763,0,B,VAL,135,136,0.000000,V,B
2,143.25,1.590,11.936,57.397,B,VAL,136,136,,136 VAL,...,CA,P40763,1,A,VAL,135,136,0.000000,V,B
3,143.25,1.590,11.936,57.397,B,VAL,136,136,,136 VAL,...,CA,P40763,0,B,VAL,135,136,0.000000,V,B
4,135.35,0.496,13.005,57.404,B,VAL,136,136,,136 VAL,...,C,P40763,1,A,VAL,135,136,0.000000,V,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17166,146.01,-51.277,5.549,0.199,A,PRO,689,689,,689 PRO,...,C,P40763,1067,A,PRO,688,689,0.983333,P,A
17167,126.95,-52.140,4.905,-0.376,A,PRO,689,689,,689 PRO,...,O,P40763,1067,A,PRO,688,689,0.983333,P,A
17168,162.87,-51.885,6.336,2.480,A,PRO,689,689,,689 PRO,...,CB,P40763,1067,A,PRO,688,689,0.983333,P,A
17169,167.82,-51.103,6.567,3.722,A,PRO,689,689,,689 PRO,...,CG,P40763,1067,A,PRO,688,689,0.983333,P,A


In [12]:

rows=[]
st = iotbx.pdb.input(file_name=pdb_id_sc+".pdb")
for atm in st.atoms_with_labels():
    try:
        if (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0]))>0:
            atm.set_b(float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            print (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            #if (mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['resname'].values[0])!=atm.resname:
            #    print ('Something is fishy!')
        else:
            atm.set_b(0)
    except:
        atm.set_b(0)


1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
10.0
10.0
10.0
10.0
10.0
10.0
10.0
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
10.0
10.0
10.0
10.0
10.0
10.0
10.0
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
28.333333333333332
28.333333333333332
28.333333333333332
28.333333333333332
28.333333333333332
28.3333

96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
70.0
70.0
70.0
70.0
70.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
68.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
63.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
36.66666666666666

31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.333

38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
26.66666666666667
26.66666666666667
26.66666666666667
26.66666666666667
26.66666666666667
26.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
60.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333

11.66666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
15.0
15.0
15.0
15.0
15.0
15.0
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
45.0
45.0
45.0
45.0
45.0
55.00000000000001
55.00000000000001
55.00000000000001
55.00000000000001
66.66666666666666
66.66666666666666
66.666666

...

1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.3333333333333335
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.3333333333

11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
43.33333333333333
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
15.0
15.0
15.0
15.0
15.0
15.0
15.0
10.0
10.0
10.0
10.0
10.0
10.0
3.3333333333333335
3.3333333333333335
3.3333333333333335
3.333333333333

38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
21.66666666666667
28.333333333333332
28.333333333333332
28.333333333333332
28.333333333333332
28.333333333333332
28.333333333333332
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
13.33333333333333
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.666

5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
10.0
10.0
10.0
10.0
10.0
10.0
10.0
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
40.0
40.0
40.0
40.0
40.0
40.0
40.0
38.33333333333334
38.33333333333334
38.33333333333

73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.3333333

In [13]:
st.write_pdb_file("mapped_"+pdb_id_sc+"_everything.pdb")

In [37]:
posres=np.load('P62799.npz')
posres['arr_0']

array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, 4.00000000e+00,
        5.00000000e+00, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
        9.00000000e+00, 1.00000000e+01, 1.10000000e+01, 1.20000000e+01,
        1.30000000e+01, 1.40000000e+01, 1.50000000e+01, 1.60000000e+01,
        1.70000000e+01, 1.80000000e+01, 1.90000000e+01, 2.00000000e+01,
        2.10000000e+01, 2.20000000e+01, 2.30000000e+01, 2.40000000e+01,
        2.50000000e+01, 2.60000000e+01, 2.70000000e+01, 2.80000000e+01,
        2.90000000e+01, 3.00000000e+01, 3.10000000e+01, 3.20000000e+01,
        3.30000000e+01, 3.40000000e+01, 3.50000000e+01, 3.60000000e+01,
        3.70000000e+01, 3.80000000e+01, 3.90000000e+01, 4.00000000e+01,
        4.10000000e+01, 4.20000000e+01, 4.30000000e+01, 4.40000000e+01,
        4.50000000e+01, 4.60000000e+01, 4.70000000e+01, 4.80000000e+01,
        4.90000000e+01, 5.00000000e+01, 5.10000000e+01, 5.20000000e+01,
        5.30000000e+01, 5.40000000e+01, 5.50000000e+01, 5.600000